In [1]:
from model_v2 import build_lm_classifier_inference, LSTM_SAVED_STATE
from utils import get_batch_classifier_inference, clean_text
import json
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline

In [2]:
with open('intent_102/checkpoints/model_configs.json', 'r') as inp:
    lm_params = json.load(inp)
with open('intent_102/checkpoints/classifier_configs.json', 'r') as inp:
    cls_params = json.load(inp)
language_model, classifier = build_lm_classifier_inference(lm_params, cls_params)

In [3]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)
saver = tf.train.Saver([x for x in tf.global_variables() if x not in tf.get_collection(LSTM_SAVED_STATE)])
saver.restore(session, 'intent_102/checkpoints/classifier_interest_test/model.cpkt-150')

INFO:tensorflow:Restoring parameters from intent_102/checkpoints/classifier_interest_test/model.cpkt-150


In [4]:
with open('IntentDetection/word2idx.json', 'r') as inp:
    word2idx = json.load(inp)
with open('IntentDetection/char2idx.json', 'r') as inp:
    char2idx = json.load(inp)
word2char = {w: [char2idx[c] for c in w] for w in word2idx}

In [5]:
def predict(texts, bsz=32):
    texts = [clean_text(x.strip()) for x in texts]
    texts = np.array([[word2char.get(w, word2char['<UNK>']) for w in sent.split()] for sent in texts])
    results = []
    for chars, lens, char_lens in get_batch_classifier_inference(texts, bsz):
        probs = session.run(classifier.probs, feed_dict={
            language_model.inputs: chars, language_model.seq_lens: lens,
            language_model.char_lens: char_lens, language_model.bptt: 20
        })
        results.append(probs)
    return np.concatenate(results, axis=0)

In [6]:
data = pd.read_csv('IntentDetection/total_purchase_text - totalpurchase_text.csv', names=['label', 'mention', '1', '2'], skiprows=1)[['label', 'mention']]

In [7]:
data.dropna(inplace=True)
data['interest'] = data['label'].map(lambda x: int('I' in x))

In [27]:
data['pred'] = np.argmax(predict(data['mention']), axis=1)

In [28]:
data[data['interest'] != data['pred']]

,label,mention,interest,pred
53,W,"Số 2 có mẫu mới này k ạ, mua ở đâu đc ạ",0,1
54,P,Similac số 3 mẫu mới ntn và giá bn vậy ạ?,0,1
57,"P,T",Số 3 giá ntn và bao giờ có mẩu mới ah,0,1
61,"P,A","Cho mình hỏi,giá vẫn vậy hay sao?mẫu này đã có...",0,1
62,P,"M cần mua số 2 loại hộp lớn nhất, báo giá m nhé",0,1
64,W,Có thể mua ản phẩm mới Similac IQ Plus ở đâu v...,0,1
65,W,"Em cũng muốn mua sữa Similac này cho bé, có th...",0,1
80,P,Minh mua 2 hop cho be 3 tuoi nhieu tien loai 9...,0,1
82,O,"Đợt rày thấy hệ tiêu hóa của bé nhà ko tốt, đa...",0,1
85,"P,W","Không biết Similac có giá thế nào nhỉ, có thể ...",0,1
